###Created CNN model and optimize it using Keras Tuner

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 2s 0us/step


In [4]:
#Scaling images to 0 and 1
train_images = train_images/255.0
test_images = test_images/255.0

In [5]:
train_images[0].shape

(28, 28)

In [6]:
#Reshape the images
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [41]:
#creating funtion to build model.

def build_model(hp):
    model_cnn = keras.Sequential([
        keras.layers.Conv2D(
        filters = hp.Int('conv_1_filter',min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
        activation = 'relu',
        input_shape=(28,28,1)
        ),
        keras.layers.Conv2D(
        filters = hp.Int('conv_2_filter',min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
        activation = 'relu'),
        keras.layers.Flatten(),
        keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])
    
    model_cnn.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                     loss = 'sparse_categorical_crossentropy',
                     metrics = ['accuracy'])
    
    return model_cnn

In [34]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [35]:
# creating function to search optimize parameters
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials=5,
                            directory='output',
                            project_name='Mnist Fashion')

In [36]:
# running search function to find best parameters
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.2)

Trial 5 Complete [00h 04m 20s]
val_accuracy: 0.8479999899864197

Best val_accuracy So Far: 0.9072499871253967
Total elapsed time: 00h 24m 25s
INFO:tensorflow:Oracle triggered exit


In [43]:
model_cnn = tuner_search.get_best_models(num_models=1)[0]

In [44]:
model_cnn.fit(train_images,train_labels,epochs=10,validation_split=0.2,initial_epoch=3)

Epoch 4/10
1500/1500 [==============================] - 87s 58ms/step - loss: 0.1410 - accuracy: 0.9474 - val_loss: 0.2563 - val_accuracy: 0.9133
Epoch 5/10
1500/1500 [==============================] - 88s 59ms/step - loss: 0.0991 - accuracy: 0.9630 - val_loss: 0.3165 - val_accuracy: 0.9131
Epoch 6/10
1500/1500 [==============================] - 92s 61ms/step - loss: 0.0706 - accuracy: 0.9736 - val_loss: 0.3473 - val_accuracy: 0.9082
Epoch 7/10
1500/1500 [==============================] - 90s 60ms/step - loss: 0.0487 - accuracy: 0.9824 - val_loss: 0.4062 - val_accuracy: 0.9084
Epoch 8/10
1500/1500 [==============================] - 89s 59ms/step - loss: 0.0366 - accuracy: 0.9869 - val_loss: 0.4362 - val_accuracy: 0.9119
Epoch 9/10
1500/1500 [==============================] - 91s 61ms/step - loss: 0.0316 - accuracy: 0.9888 - val_loss: 0.4931 - val_accuracy: 0.9048
Epoch 10/10
1500/1500 [==============================] - 90s 60ms/step - loss: 0.0269 - accuracy: 0.9903 - val_loss: 0.5679 

In [45]:
from tensorflow.keras.models import load_model

model_cnn.save('CNN_kerastuner.h5')